In [1]:
def spawn_agent(agent_id, create_env, gen_act, mb, n_episodes=4, max_steps=500, render=False):
    env = create_env()
    for episode in range(n_episodes):
        obs = env.reset()
        mb.start_rollout(agent_id)
        for step in range(max_steps):
            act = gen_act([obs])
            
            if render:
                env.render()
                time.sleep(0.02)
                
            obs_next, rew, d, _ = env.step(act)
            
            mb.record(agent_id, obs, act, rew)
            obs = obs_next
            
            if d:
                break

In [2]:
def gather_data(mpmb, n_threads=7, n_episodes=4, return_rewards=False, reset_mem=True):
    agent_pool = []
    
    for i in range(n_threads):
        agent_pool.append(Thread(target=spawn_agent, args=(i, make_lunar_lander_c, network.gen_act, mpmb),
                                                     kwargs={'n_episodes': math.ceil(n_episodes/n_threads)}))
        agent_pool[-1].start()

    for agent in agent_pool:
        agent.join()
        
    
    if return_rewards:
        return mpmb.get_avg_reward(), mpmb.to_data(reset=reset_mem)
    return mpmb.to_data(reset=reset_mem)

In [3]:
# mpmb = MTMemoryBuffer()
# ep_rewards = []

# for i in range(n_episodes//update_freq):
# #     ep_reward, train_data = gather_data(mpmb, n_episodes=update_freq, return_rewards=True)
# #     ep_rewards.append(ep_reward)
#     spawn_agent(0, make_cart_pole, network.gen_act, mpmb, n_episodes=update_freq, max_steps=300, render=False)
#     print(mpmb.get_avg_reward())
#     train_data = mpmb.to_data()
#     network.train(train_data)
# #     if i % print_freq == 0 and i != 0:
# #         print(f'Update #{int(i)}, Recent Reward:', np.mean(ep_rewards[print_freq:]))